# Exercise 4:  Reusing Stored Model in Web App

## Load and Test the Model from a File

In [171]:
# For serialization and deserialization of data from/to file
# from sklearn.externals 
import joblib
import pandas as pd
import numpy as np

## Install Flask

In [172]:
# !pip install flask-bootstrap
# !pip install wtforms

### Import model

In [173]:
gaussian_model = joblib.load('../../saved_models/popularity_prediction_model.pkl')

### Spotify feature import

In [174]:
#Spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

spotify_client_id = "3a2f65eb20164b24b59b7fdd6876bb51"
spotify_client_secret = "9f6d3a070cdf4eb8adda14a794d60469"

#Genius credentials
genius_token = "4SoLPv5UHyFZyAe5T-PhxS-bRCRrdNIgMNpeqjjIaWvnRbUmTpoGxd9knxBtlQyg"

#Authentication spotify - without user
client_credentials_manager = SpotifyClientCredentials(client_id="3a2f65eb20164b24b59b7fdd6876bb51", client_secret="9f6d3a070cdf4eb8adda14a794d60469")
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### Define a function for making predictions on songs popularity.

In [186]:
def analyseSongPopularity(songUrl):
    # Get song by songUrl.
    features = sp.audio_features(songUrl)[0]
    # Convert to dataframe.
    df = pd.DataFrame(features, index=['i',])
    # Get values and remove unwanted columns.
    final_set = df.values[:,0:10]
    # Predict popularity of song
    return gaussian_model.predict(final_set)

### Try the function with songs

In [193]:
analyseSongPopularity('https://open.spotify.com/track/6OtCIsQZ64Vs1EbzztvAv4?si=2f482142d5ca42b5')

[[0.634 0.69 6 -7.804 1 0.052 0.0771 0 0.132 0.645]]


array(['Popular'], dtype='<U11')

In [194]:
analyseSongPopularity('https://open.spotify.com/track/0HcTnHDZH8tmf2YbUiMgO2?si=25c97d0c35914864')

[[0.567 0.622 11 -6.713 0 0.0356 0.0119 2.31e-05 0.108 0.297]]


array(['Not Popular'], dtype='<U11')

## Create Flask Web Server

In [191]:
%%writefile mswebapp.py

# Import libraries
import numpy as np

# from sklearn.externals 
import joblib
import pickle
from flask import Flask, request, jsonify, render_template
import pandas as pd
import json
import plotly
import plotly.express as px
from sklearn.decomposition import PCA

# Create An Instance (Of the application)
app = Flask(__name__, template_folder='./templates')

@app.route('/barchart')
def barchart():
   df = pd.DataFrame({
      'Fruit': ['Apples', 'Oranges', 'Bananas', 'Apples', 'Oranges', 
      'Bananas'],
      'Amount': [4, 1, 2, 2, 4, 5],
      'City': ['SF', 'SF', 'SF', 'Montreal', 'Montreal', 'Montreal']
   })
   fig = px.bar(df, x='Fruit', y='Amount', color='City', 
      barmode='group')
   graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
   return render_template('visual.html', graphJSON=graphJSON)

@app.route('/piechart')
def piechart():
   df = pd.DataFrame({
      'Fruit': ['Apples', 'Oranges', 'Bananas', 'Melons', 'Cherries', 
      'Pinapples'],
      'Amount': [4, 1, 2, 2, 4, 5],
   })
   fig = px.pie(df, names='Fruit', values='Amount')
   graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
   return render_template('visual.html', graphJSON=graphJSON)

@app.route('/scatterplot')
def scatterplot():
   df = pd.DataFrame({
      'x': [0, 1, 2, 5, 7, 10, 12, 15, 19, 24],
      'y': [4, 2, 7, 15, 11, 19, 3, 5, 9, 8]
   })
   fig = px.scatter(df, x='x', y='y')
   graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
   return render_template('visual.html', graphJSON=graphJSON)

@app.route('/pcascatter')
def pcascatter():
    df = px.data.iris()
    X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

    pca = PCA(n_components=2)
    components = pca.fit_transform(X)

    fig = px.scatter(components, x=0, y=1, color=df['species'], labels={'0': 'X-Value', '1': 'Y-Value'})
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return render_template('visual.html', graphJSON=graphJSON)

if __name__ == '__main__':
    app.run(debug=True)

Overwriting mswebapp.py


In [ ]:
!python mswebapp.py